## Data Preparation

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
dataset, dataset_info = tfds.load('malaria', with_info=True, as_supervised=True, split=['train'], shuffle_files=True)

TRAIN_RATIO = 0.6
VAL_RATIO = 0.2
TEST_RATIO = 0.2

def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  DATASET_SIZE = len(dataset)
  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))
  return train_dataset, val_dataset, test_dataset

train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

IM_SIZE = 224
def resize_rescale(image, label):
  return tf.image.resize(image, (IM_SIZE, IM_SIZE))/255.0, label

train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8).batch(BATCH_SIZE)
test_dataset = test_dataset.shuffle(buffer_size=8).batch(BATCH_SIZE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating train examples...:   0%|          | 0/27558 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incompleteF1U6VM/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


## Custom Layer

- In Dense(1), we don't need to specify the feature number
- If input is (B,F), (F,1) for weight is done automatically as we put the input

In [ ]:
class CustomDense(Layer):
  def __init__(self, output_units, activation):
    super(CustomDense, self).__init__()
    self.output_units = output_units
    self.activation = activation

  def build(self, input_feature_shape):
    self.w = self.add_weight(shape=(input_feature_shape[-1], self.output_units), initializer="random_normal", trainable=True) # inherited from Layer
    self.b= self.add_weight(shape=(self.output_units,), initializer="random_normal", trainable=True)

  def call(self, input_features):
    pre_output = tf.matmul(input_features, self.w) + self.b
    if self.activation == "relu":
      return tf.nn.relu(pre_output) # (B,F) * (F,1)
    elif self.activation == "sigmoid":
      return tf.math.sigmoid(pre_output)
    else:
      return pre_output

In [ ]:
custom_model = tf.keras.Sequential([InputLayer(input_shape=(IM_SIZE, IM_SIZE, 3)),

                             Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu'),
                             BatchNormalization(),
                             MaxPool2D(pool_size=2, strides=2),

                             Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu'),
                             BatchNormalization(),
                             MaxPool2D(pool_size=2, strides=2),

                             Flatten(),

                             CustomDense(100, activation='relu'),
                             BatchNormalization(),
                             CustomDense(10, activation='relu'),
                             BatchNormalization(),
                             CustomDense(1, activation='sigmoid')]) # 0 or 1
print(custom_model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 222, 222, 6)       168       
                                                                 
 batch_normalization_46 (Bat  (None, 222, 222, 6)      24        
 chNormalization)                                                
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 111, 111, 6)      0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 109, 109, 16)      880       
                                                                 
 batch_normalization_47 (Bat  (None, 109, 109, 16)     64        
 chNormalization)                                                
                                                      

In [ ]:
custom_model.compile(optimizer=Adam(learning_rate=0.001),
              loss=BinaryCrossentropy(),
              metrics=['accuracy'])
history = custom_model.fit(train_dataset, validation_data=val_dataset, epochs=5, verbose=1)

Epoch 1/5
517/517 [==============================] - 45s 76ms/step - loss: 0.6247 - accuracy: 0.6591 - val_loss: 0.7309 - val_accuracy: 0.5680
Epoch 2/5
517/517 [==============================] - 41s 79ms/step - loss: 0.5814 - accuracy: 0.7082 - val_loss: 0.8283 - val_accuracy: 0.5578
Epoch 3/5
517/517 [==============================] - 45s 87ms/step - loss: 0.5461 - accuracy: 0.7370 - val_loss: 0.6013 - val_accuracy: 0.6453
Epoch 4/5
517/517 [==============================] - 39s 76ms/step - loss: 0.4627 - accuracy: 0.7878 - val_loss: 0.5403 - val_accuracy: 0.7229
Epoch 5/5
517/517 [==============================] - 39s 75ms/step - loss: 0.3888 - accuracy: 0.8339 - val_loss: 0.3730 - val_accuracy: 0.8498
